This is a small classification problem. Now we use RandomizedSearchCV to seach good hyperparameters. Goal is demonstration only.

ref: https://github.com/gecso75/data/blob/main/xAPI-Edu-Data-desc.txt

In [1]:
import requests
import io
import numpy as np
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

Using TensorFlow backend.


In [2]:
url = "https://raw.githubusercontent.com/gecso75/data/main/xAPI-Edu-Data.csv"
download = requests.get(url).content
df = read_csv(io.StringIO(download.decode('utf-8')))

In [3]:
df.head()

,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays,Class
0,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,15,16,2,20,Yes,Good,Under-7,M
1,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,20,20,3,25,Yes,Good,Under-7,M
2,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,10,7,0,30,No,Bad,Above-7,L
3,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,30,25,5,35,No,Bad,Above-7,L
4,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,40,50,12,50,No,Bad,Above-7,M


In [4]:
df.dtypes

gender                      object
NationalITy                 object
PlaceofBirth                object
StageID                     object
GradeID                     object
SectionID                   object
Topic                       object
Semester                    object
Relation                    object
raisedhands                  int64
VisITedResources             int64
AnnouncementsView            int64
Discussion                   int64
ParentAnsweringSurvey       object
ParentschoolSatisfaction    object
StudentAbsenceDays          object
Class                       object
dtype: object

We are not one-hot encoding our label here as we are using KerasClassifier.

In [5]:
X = df.copy()
y = X.pop('Class')

this can be done on the whole set:

In [6]:
trans1 = ColumnTransformer([
    ('tr1', OneHotEncoder(drop='first'), [0, 1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 15])],
    remainder='passthrough',
    sparse_threshold=0
)
X = trans1.fit_transform(X)

this will be done in randomizedsearchcv call:

In [7]:
trans2 = ColumnTransformer([
    ('tr1', StandardScaler(), [-1, -2, -3, -4])],
    remainder='passthrough'
)

In [8]:
def make_model(dense_layer_sizes, activation, optimizer, droprate):
    model = Sequential()

    for i, layer_size in enumerate(dense_layer_sizes):
        if i == 0:
            model.add(Dense(units=layer_size, input_dim=60, activation=activation))
            model.add(Dropout(rate=droprate))
        else:
            model.add(Dense(units=layer_size, activation=activation))
            model.add(Dropout(rate=droprate))

    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy')
    return model

In [9]:
my_classifier = KerasClassifier(make_model)
my_pipe = Pipeline([['onehot', trans2], ['classifier', my_classifier]])

In [10]:
param_distributions = dict(
    classifier__dense_layer_sizes = [[30], [60], [90], [120],
                        [30, 30], [60, 60], [90, 90], [120, 120],
                        [30, 15, 30], [60, 30, 60], [90, 45, 90], [120, 60, 120]],
    classifier__activation=['linear', 'sigmoid', 'hard_sigmoid', 'tanh', 'softsign', 'relu', 'elu', 'selu', 'softplus'],
    classifier__optimizer=['sgd', 'adagrad', 'rmsprop', 'adadelta', 'adam', 'adamax', 'nadam'],
    classifier__droprate=[0.0, 0.1, 0.2, 0.3, 0.4],
    classifier__epochs=[100],
    classifier__batch_size=[8, 16]
)

In [11]:
validator = RandomizedSearchCV(my_pipe, param_distributions, scoring='accuracy', cv=3, n_jobs=-1, n_iter=5)

In [12]:
validator.fit(X, y)

Epoch 1/100
480/480 [==============================] - 0s 254us/step - loss: 1.0831
Epoch 2/100
480/480 [==============================] - 0s 62us/step - loss: 0.9371
Epoch 3/100
480/480 [==============================] - 0s 54us/step - loss: 0.8508
Epoch 4/100
480/480 [==============================] - 0s 58us/step - loss: 0.7959
Epoch 5/100
480/480 [==============================] - 0s 54us/step - loss: 0.7445
Epoch 6/100
480/480 [==============================] - 0s 56us/step - loss: 0.7224
Epoch 7/100
480/480 [==============================] - 0s 56us/step - loss: 0.6838
Epoch 8/100
480/480 [==============================] - 0s 56us/step - loss: 0.6712
Epoch 9/100
480/480 [==============================] - 0s 55us/step - loss: 0.6649
Epoch 10/100
480/480 [==============================] - 0s 57us/step - loss: 0.6361
Epoch 11/100
480/480 [==============================] - 0s 56us/step - loss: 0.6293
Epoch 12/100
480/480 [==============================] - 0s 56us/step - loss: 0.6095


480/480 [==============================] - 0s 56us/step - loss: 0.4274
Epoch 99/100
480/480 [==============================] - 0s 54us/step - loss: 0.4489
Epoch 100/100
480/480 [==============================] - 0s 61us/step - loss: 0.4455


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[['onehot',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='passthrough',
                                                                sparse_threshold=0.3,
                                                                transformer_weights=None,
                                                                transformers=[('tr1',
                                                                               StandardScaler(copy=True,
                                                                                              with_mean=True,
                                                                                              with_std=True),
                                                                               [-1

In [13]:
print(validator.best_score_)
print(validator.best_params_)
print(validator.best_estimator_)

0.71875
{'classifier__optimizer': 'adadelta', 'classifier__epochs': 100, 'classifier__droprate': 0.2, 'classifier__dense_layer_sizes': [120], 'classifier__batch_size': 16, 'classifier__activation': 'sigmoid'}
Pipeline(memory=None,
         steps=[('onehot',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('tr1',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  [-1, -2, -3, -4])],
                                   verbose=False)),
                ['classifier',
         verbose=False)
